In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
ds = pd.read_csv('./titanic.csv')
ds2 = pd.read_csv('./test.csv')

In [5]:
ds2.head(n=10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [6]:
mean_Age = ds2['Age'].mean()
ds2['Age'] = ds2['Age'].fillna(int(mean_Age))

sex_mapping = {
    'male': 0,
    'female': 1
}

sex = ds2['Sex'].values
quant_sex2 = np.zeros(sex.shape)

for ix in range(quant_sex2.shape[0]):
    quant_sex2[ix] = sex_mapping[sex[ix]]

In [7]:
mean_Age = ds['Age'].mean()
ds['Age'] = ds['Age'].fillna(int(mean_Age))

sex_mapping = {
    'male': 0,
    'female': 1
}

sex = ds['Sex'].values
quant_sex = np.zeros(sex.shape)

for ix in range(quant_sex.shape[0]):
    quant_sex[ix] = sex_mapping[sex[ix]]

In [8]:
print quant_sex.dtype, quant_sex2.dtype

float64 float64


In [9]:
data = np.asarray([ds['Age'], ds['Pclass'], ds['SibSp'], ds['Parch'], ds['Fare'], quant_sex, ds['Survived']]).astype("float").T

test_data = np.asarray([ds2['Age'], ds2['Pclass'], ds2['SibSp'], ds2['Parch'], ds2['Fare'], quant_sex2]).astype("float").T

Y = ds['Survived'].values
cols = ['Age', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Survived']
print data[0]

[ 22.     3.     1.     0.     7.25   0.     0.  ]


In [10]:
def data_split(dataset, col_id, value):
    data_right = []
    data_left = []
    for ix in range(dataset.shape[0]):
        if dataset[ix, col_id] <= value:
            data_left.append(dataset[ix, :])
        else:
            data_right.append(dataset[ix, :])
    return np.asarray(data_right), np.asarray(data_left)

In [11]:
def entropy(dataset, col=-1):
    if dataset.shape[0] == 0:
        return 0
    p = dataset[:, col].mean()
    if p == 1 or p == 0:
        return 0.0
    ent = (-1.0 * p * np.log2(p)) + (-1.0 * (1.0 - p) * np.log2(1.0 - p))
    return ent
    
def information_gain(parent_set, child_1, child_2):
    ent_p = entropy(parent_set)
    ent_c1 = entropy(child_1)
    ent_c2 = entropy(child_2)
    
    f = float(child_1.shape[0])/parent_set.shape[0]
    Ig = ent_p - (f*ent_c1 + (1-f)*ent_c2)
    return Ig

In [12]:
INF = 100000
class DT:
    def __init__(self, depth=0, max_depth=10):
        self.left = None
        self.right = None
        self.col_id = None
        self.value = None
        self.depth = depth
        self.max_depth = max_depth
        self.n_survival = None
        self.n_death = None
    
    def get_best_gain(self, dataset):
        n_cols = 6
        all_gains = []
        self.n_survival = dataset[:, -1].sum()
        self.n_death = dataset.shape[0] - self.n_survival
        for cx in range(n_cols):
            mean_val_cx = dataset[:, cx].mean()
            right, left = data_split(dataset, cx, mean_val_cx)
            total_gain = information_gain(dataset, right, left)
            all_gains.append(total_gain)
        self.col_id = np.asarray(all_gains).argmax()
        self.value = dataset[:, self.col_id].mean()
        data_right, data_left = data_split(dataset, self.col_id, self.value)
        if data_left.shape[0] > 0 and self.depth < self.max_depth:
            self.left = DT(depth=self.depth+1, max_depth=self.max_depth)
            self.left.get_best_gain(data_left)
        if data_right.shape[0] > 0 and self.depth < self.max_depth:
            self.right = DT(depth=self.depth+1, max_depth=self.max_depth)
            self.right.get_best_gain(data_right)
        return

    def predict(self, example):
        exp_val = example[self.col_id]
        if exp_val <= self.value:
            if not self.left == None:
                return self.left.predict(example)
            else:
                return self.decide()
        else:
            if not self.right == None:
                return self.right.predict(example)
            else:
                return self.decide()
    
    def decide(self):
        if self.n_survival >= self.n_death:
            return 1
        else:
            return 0


In [13]:
# dt = DT(max_depth=10)
# dt.get_best_gain(data)

In [14]:
X_train = data[:, :]
y_train = data[:, -1]

X_test = test_data

y_pred = []

dt = DT(max_depth=6)
dt.get_best_gain(X_test)
for yx in range(X_test.shape[0]):
    y_pred.append(int(dt.predict(X_test[yx, :])))

print y_pred
print type(y_pred[0])

[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

In [104]:
final_ans = np.zeros((len(y_pred), 2))
final_ans[:,0] = ds2['PassengerId']
final_ans[:,1] = y_pred
print final_ans

[[  8.92000000e+02   0.00000000e+00]
 [  8.93000000e+02   1.00000000e+00]
 [  8.94000000e+02   0.00000000e+00]
 [  8.95000000e+02   0.00000000e+00]
 [  8.96000000e+02   1.00000000e+00]
 [  8.97000000e+02   0.00000000e+00]
 [  8.98000000e+02   1.00000000e+00]
 [  8.99000000e+02   0.00000000e+00]
 [  9.00000000e+02   1.00000000e+00]
 [  9.01000000e+02   0.00000000e+00]
 [  9.02000000e+02   0.00000000e+00]
 [  9.03000000e+02   0.00000000e+00]
 [  9.04000000e+02   1.00000000e+00]
 [  9.05000000e+02   0.00000000e+00]
 [  9.06000000e+02   1.00000000e+00]
 [  9.07000000e+02   1.00000000e+00]
 [  9.08000000e+02   0.00000000e+00]
 [  9.09000000e+02   0.00000000e+00]
 [  9.10000000e+02   1.00000000e+00]
 [  9.11000000e+02   1.00000000e+00]
 [  9.12000000e+02   0.00000000e+00]
 [  9.13000000e+02   0.00000000e+00]
 [  9.14000000e+02   1.00000000e+00]
 [  9.15000000e+02   0.00000000e+00]
 [  9.16000000e+02   1.00000000e+00]
 [  9.17000000e+02   0.00000000e+00]
 [  9.18000000e+02   1.00000000e+00]
 

In [105]:
final = pd.DataFrame({'PassengerId': ds2['PassengerId'], 'Survived': y_pred})
final.to_csv('output.csv', header=True, columns = ['PassengerId', 'Survived'], sep = ',', index = False)

In [107]:
ds = pd.read_csv('./output.csv')
ds.head(n=418)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
